In [1]:
import numpy as np
import pandas as pd
import tweepy
import time
import datetime
import requests
import bs4
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
import math

In [2]:
# os.chdir(r'C:\Users\Ben\Documents\2019 Summer\TradingAlgo\Tweets')
# os.mkdir(r'C:\Users\Ben\Documents\2019 Summer\TradingAlgo\Tweets\bitcoin')

In [2]:
C_Key = 'ILYTd5Abkw85OKTd5sAbSpPdC'
C_Secret='1hHTjqyq5Kitt3b5gka6uPMkvKzuz5wO7C63HcRLP5v2mHUtz6'
A_Token='2758135350-UfWPEgJPQJTCHvCQRQEzcavyl45mcwwLkRnWWBi'
A_Token_Secret='LKa0IlzGhWdLKgzhATUtFx0kM5AGG6lAWBUoOEP1lKj9g'

auth = tweepy.OAuthHandler(consumer_key=C_Key, consumer_secret=C_Secret)
auth.set_access_token(A_Token, A_Token_Secret)

api = tweepy.API(auth)

In [4]:
chrome_option = Options()
chrome_option.add_argument(r'user-data-dir=C:\Users\Ben\AppData\Local\Google\Chrome\User Data\Default')
chrome_option.add_argument("--no-sandbox")
chrome_option.add_argument("--disable-dev-shm-usage")
chrome_option.add_argument("--headless")
chrome_option.add_experimental_option('w3c', False)
browser = webdriver.Chrome(executable_path=r'C:\Users\Ben\Documents\Varsity Surf\chromedriver_win32\chromedriver.exe',chrome_options=chrome_option)


In [ ]:
class Tweet_Collecter(object):
    
    def __init__(self, init_browser=True, browser=None):
        
        import os
        import math
        import pandas as pd
        import numpy as np
        from selenium.webdriver.chrome.options import Options
        from selenium import webdriver
        import tweepy
        import time
        import datetime
        import requests
        
        self.init_browser= init_browser
        
        self.C_Key = 'ILYTd5Abkw85OKTd5sAbSpPdC'
        self.C_Secret='1hHTjqyq5Kitt3b5gka6uPMkvKzuz5wO7C63HcRLP5v2mHUtz6'
        self.A_Token='2758135350-UfWPEgJPQJTCHvCQRQEzcavyl45mcwwLkRnWWBi'
        self.A_Token_Secret='LKa0IlzGhWdLKgzhATUtFx0kM5AGG6lAWBUoOEP1lKj9g'

        auth = tweepy.OAuthHandler(consumer_key=C_Key, consumer_secret=C_Secret)
        auth.set_access_token(A_Token, A_Token_Secret)
        self.auth = auth
        
        self.api = tweepy.API(auth)
        if init_browser:
            chrome_option = Options()
            chrome_option.add_argument(r'user-data-dir=C:\Users\Ben\AppData\Local\Google\Chrome\User Data\Default')
            chrome_option.add_argument("--no-sandbox")
            chrome_option.add_argument("--disable-dev-shm-usage")
            chrome_option.add_argument("--headless")
            self.browser = webdriver.Chrome(executable_path=r'C:\Users\Ben\Documents\Varsity Surf\chromedriver_win32\chromedriver.exe',
                                            chrome_options=chrome_option)
        else:
            self.browser = browser
    
    def collect_tweets(self, filters = ['#btc', 'bitcoin'], start_date=datetime.datetime.today()-datetime.timedelta(days=1),
                       end_date= datetime.datetime.today(), approx_tweets_per_interval = 200, interval_mins=20,
                      save_folder_name='temp', filename='bitcoin', round_interval = True):
    
        browser = self.browser
        api = self.api
        
        if round_interval:
            start_date = start_date - datetime.timedelta(minutes=start_date.minute % interval_mins,
                                 seconds= start_date.second,
                                 microseconds=start_date.microsecond)

        if start_date>end_date:
            raise Exception('Start date must be after end date')

        print('Calculating tweet ids to be collected...')
        ################### Find id of tweet exactly 24 hours later
        since_datetime = start_date.date()
        since_datetime = datetime.datetime(since_datetime.year, since_datetime.month, since_datetime.day)
        until_datetime = (start_date + datetime.timedelta(days=1)).date()
        until_datetime = datetime.datetime(until_datetime.year, until_datetime.month, until_datetime.day)
        end_datetime = end_date.date()
        end_datetime = datetime.datetime(end_datetime.year, end_datetime.month, end_datetime.day)

        since_date = since_datetime.isoformat()[:10]
        until_date = until_datetime.isoformat()[:10]


        filters = [fil.replace('#','%23') for fil in filters]
        self.filters = filters
        query_filters = str('%20OR%20'.join(filters))
        query = query_filters + '%20since%3A'+since_date+'%20until%3A'+until_date+'&src=typd'

        base_url = 'https://twitter.com/search?f=tweets&q='
        url = base_url + query
        browser.get(url) 
        time.sleep(1)


        endtweet_id = int(browser.find_elements_by_class_name('js-actionable-tweet')[0].get_attribute('data-tweet-id'))

        #################### Find id of last tweet before timeperiod starts

        until_datetime2 = start_date.date()
        until_datetime2 = datetime.datetime(until_datetime2.year, until_datetime2.month, until_datetime2.day)
        until_date2 = until_datetime2.isoformat()[:10]

        query = query_filters + '%20until%3A'+until_date2+'%20lang%3Aen&src=typd'

        url = base_url + query

        browser.get(url) 
        time.sleep(1)

        starttweet_id = int(browser.find_elements_by_class_name('js-actionable-tweet')[0].get_attribute('data-tweet-id'))

        #################### calculate tweet id pairs needed to collect correct tweets at each sub-timeperiod


        id_interval = (endtweet_id - starttweet_id)/((60*24)/interval_mins)

        ##calculate which id to start on and end on
        ##start tweet
        start_timejump = start_date - since_datetime #difference between time specified to start at and time at which
                                               #starttweet_id currently specifies to collect from
        start_intervalsjump = int(math.ceil(start_timejump.total_seconds()/(60*interval_mins)))


        total_timeperiod_length = end_date - start_date #total length of time over whole specified timeperiod
                                                        # enables calculation of how many id intervals need to be collected

        num_intervals= math.ceil(total_timeperiod_length.total_seconds()/(60*interval_mins)) #number of intervals to collect

        start_id = int(starttweet_id + start_intervalsjump * id_interval)

        since_ids= [int(start_id + i*id_interval) for i in range(num_intervals)]
        max_ids = [int(x + id_interval) for x in since_ids]
        
        time_range = pd.date_range(start = start_date, end = end_date, freq = str(interval_mins)+'min')

        print('Tweet ids calculated!')

        ################## Start tweet collection
        
        ## initialise class for scrolling webpage
        class wait_for_more_than_n_elements_to_be_present(object):
                def __init__(self, locator, count):
                    self.locator = locator
                    self.count = count

                def __call__(self, browser):
                    try:
                        elements = EC._find_elements(browser, self.locator)
                        return len(elements) > self.count
                    except StaleElementReferenceException:
                        return False
        
        
        filepath_stem = r'C:\Users\Ben\Documents\2019 Summer\TradingAlgo\Tweets\ '.strip() + save_folder_name + r'\ '.strip()       
        if not os.path.exists(filepath_stem):
            os.mkdir(filepath_stem)
    
        print('Beginning tweet collection:')
        iteration_counter=1

        for since_id, max_id, timestamp in zip(since_ids,max_ids, time_range): # for each subtimeperiod/interval of tweets

            print('Collecting tweets from interval:',iteration_counter)
            iteration_counter+=1

            query = query_filters + '%20since_id%3A'+str(int(since_id))+'%20max_id%3A'+str(int(max_id))+'&src=typd'
            url = base_url + query

                    
            browser.get(url)
            
            wait = WebDriverWait(browser, 10)
            try:
                # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':
                wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li[data-item-id]")))

                # scroll down to the last tweet until there are no more tweets:
                number_of_tweets=0
                while number_of_tweets<approx_tweets_per_interval:

                    # extract all the tweets:
                    tweets = browser.find_elements_by_css_selector("li[data-item-id]")

                    # find number of visible tweets:
                    number_of_tweets = len(tweets)

                    # keep scrolling:
                    browser.execute_script("arguments[0].scrollIntoView();", tweets[-1])

                    try:
                        # wait for more tweets to be visible:
                        wait.until(wait_for_more_than_n_elements_to_be_present(
                            (By.CSS_SELECTOR, "li[data-item-id]"), number_of_tweets))
                    
                    except TimeoutException:
                        # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:
                        print(number_of_tweets)
                        break

                # extract the html for the whole lot:
                page_source = browser.page_source

            except TimeoutException:
                print('Likely issue: No tweets from query')
                
            # Extract data from webpage
            soup = bs4.BeautifulSoup(page_source)

            tweets = []
            for li in soup.find_all("li", class_='js-stream-item'):

                # If our li doesn't have a tweet-id, we skip it as it's not going to be a tweet.
                if 'data-item-id' not in li.attrs:
                    continue

                else:
                    tweet = {
                        'tweet_id': li['data-item-id'],
                        'text': None,
                        'user_id': None,
                        'user_screen_name': None,
                        'user_name': None,
                        'created_at': None,
                        'retweets': 0,
                        'likes': 0,
                        'replies': 0
                    }

                # Tweet Text
                text_p = li.find("p", class_="tweet-text")
                if text_p is not None:
                    tweet['text'] = text_p.get_text()

                # Tweet User ID, User Screen Name, User Name
                user_details_div = li.find("div", class_="tweet")
                if user_details_div is not None:
                    tweet['user_id'] = user_details_div['data-user-id']
                    tweet['user_screen_name'] = user_details_div['data-screen-name']
                    tweet['user_name'] = user_details_div['data-name']

                # Tweet date
                date_span = li.find("span", class_="_timestamp")
                if date_span is not None:
                    tweet['created_at'] = float(date_span['data-time-ms'])

                # Tweet Retweets
                retweet_span = li.select("span.ProfileTweet-action--retweet > span.ProfileTweet-actionCount")
                if retweet_span is not None and len(retweet_span) > 0:
                        tweet['retweets'] = int(retweet_span[0]['data-tweet-stat-count'])

                # Tweet Likes
                like_span = li.select("span.ProfileTweet-action--favorite > span.ProfileTweet-actionCount")
                if like_span is not None and len(like_span) > 0:
                    tweet['likes'] = int(like_span[0]['data-tweet-stat-count'])

                # Tweet Replies
                reply_span = li.select("span.ProfileTweet-action--reply > span.ProfileTweet-actionCount")
                if reply_span is not None and len(reply_span) > 0:
                    tweet['replies'] = int(reply_span[0]['data-tweet-stat-count'])

                tweets.append(tweet)

            ### convert data to dataframe

            tweets_df= pd.DataFrame(data=tweets)

            #tweets_df['followers'] = tweets_df['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])
            savefile = filepath_stem + filename + timestamp.isoformat()[:19].replace(':','') +'.csv'

            tweets_df.to_csv(savefile)

        print('Finished!')
        return self

In [ ]:
tmp = Tweet_Collecter(init_browser = False,browser=browser)
tmp = tmp.collect_tweets(start_date=datetime.datetime(2019,6,1), end_date= datetime.datetime(2019,7,31),
                         approx_tweets_per_interval=5000, interval_mins=60,filename='bitcoin',
                         save_folder_name='bitcoin', filters=['#btc','bitcoin'])


Calculating tweet ids to be collected...
Tweet ids calculated!
Beginning tweet collection:
140


C:\Users\Ben\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Ben\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


1093
1087
892
962
835
891
832
1047
1011
1151
1067
1231
1237
1302
1305
1358
1261
1276
1166
1163
1170
1049
1003
931
866
868
847
821
767
760
945
962
996
1035
1029
1268
1358
1275
1179
1160
1167
1176
236
1060
1026
1074
331
957
898
863
968
170
1155
1150
1136
1144
1246
1249
1338
523
1557
1452
498
1536
199
1446
1350
1376
1129
1179
1225
1664
1097
974
887
1076
1054
1097
1205
1226
1178
794
1216
1454
1303
1393
1430
1424
1614
1562
1422
1389
1230
1204
1101
1054
996
986
851
950
1003
1045
1148
1202
1135
1209
1333
215
1517
1652
1582
1468
1546
1431
1297
1421
1222
1198
1003
943
1002
914
950
949
1047
1087
1178
1085
1055
1207
1309
1323
1501
1504
1416
1452
1395
1497
1330
1240
1153
1035
971
878
939
794
806
874
828
1063
1294
1151
684
1221
1311
1355
1473
1503
1509
1526
1422
568
1466
1343
320
379
1080
886
901
769
694
770
838
783
1075
901
907
1007
967
1034
1088
1139
1109
1064
1090
1052
1052
949
924
857
768
792
742
715
687
735
760
741
869


814
396
951
900
1041
1134
1152
1086
1182
1046
1121
1090
995
944
930
816
741
894
836
751
749
894
118
1133
1041
1063
476
1467
1566
1538
1566
1464
316
827
1292
1303
1223
1061
1058
1051
1008
926
814
811
906
874
923
1095
1110
1108
1267
1256
180
1342
1457
1368
1385
1373
1341
1266
1321
1190
988
1051
884
911
891
882
883
520
928
886
1139
1088
18
1300
1436
1354
1542
1895
1553
1470
1417
1242
140
1220
615
1023
1014
869
870
856
944
140
1000
1230
1233
1190
1230
1431
1463
1718
1834
1645
1630
1448
1325
1385
1316
1368
1238
1068
1002
940
876
40
974
980
1034
1206
1146
658
1252
1258
1486
1837
1734
1716
1542
1316
1291
1246
1359
1557
1337
1090
1083
940
935
837
839
926
903
989
1050
1021
983
1010
1225
1218
1242
931
1360
676
1233
1129
1065
1070
1001
990
962
1189
1138
1080
1074
635
1586
1468
958
1209
1222
46
1306
1419
1626
1560
1457
1521
1172
1240
1266
1019
1275
1201
1225
1160
1182
1000
40
1009
1129
1238
1367
805
879
1784
635


1960
1933
1398
80
1850
1687
1638
1656
1720
1475
1338
1200
1162
1206
1050
1005
1164
1196
1305
79
1731
1879
1989
2203
2414
2300
2226
2235
2044
1885
1772
1706
1509
1402
1363
1145
1119
1192
1092
760
1149
1277
1472
1430
1362
1418
938
1685
1791
1688
1694
1758
1553
1490
1372
1448
1345
1393
1287
1176
1112
1065
1097
1087
1102
1086
1395
1319
1341
1314
1384
797
1038
1758
1789
1837
1596
1530
1634
2003
680
1413
1187
1230
1616
1684
399
1254
1361
1360
637
1588
1520
1699
1766
553
2273
1961
2003
1975
620
2004
1777
1604
1604
1437
3178
2838
240
2700
2240
1804
1849
1837
1969
1786
1080
1000
2094
3158
380
400
2619
2442
397
1963
1787
1646
1603
1465
1359
1354
1150
1150
817
1079
1097
1095
1267
1228
219
1399
1404
1552
1676
1662
1731
1638
1578
1503
1706
1682
250
1385
1292
1298
1281
1097
1156
57
1299
1331
1573
1710
1701
1643
498
2099
2107
2050
2124
2179
2029
1275
1691
1588
1589
760
1603
1318
1281
1369
1253


In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException



driver = browser
url1 = 'https://twitter.com/search?f=tweets&q=btc%20OR%20bitcoin%20since%3A2015-05-01%20until%3A2015-05-02&src=typd' 
driver.get(url1)


class wait_for_more_than_n_elements_to_be_present(object):
    def __init__(self, locator, count):
        self.locator = locator
        self.count = count
 
    def __call__(self, driver):
        try:
            elements = EC._find_elements(driver, self.locator)
            return len(elements) > self.count
        except StaleElementReferenceException:
            return False
wait = WebDriverWait(driver, 10)
try:
    # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li[data-item-id]")))

    # scroll down to the last tweet until there are no more tweets:
    number_of_tweets=0
    while number_of_tweets<200:

        # extract all the tweets:
        tweets = driver.find_elements_by_css_selector("li[data-item-id]")

        # find number of visible tweets:
        number_of_tweets = len(tweets)

        # keep scrolling:
        driver.execute_script("arguments[0].scrollIntoView();", tweets[-1])

        try:
            # wait for more tweets to be visible:
            wait.until(wait_for_more_than_n_elements_to_be_present(
                (By.CSS_SELECTOR, "li[data-item-id]"), number_of_tweets))
            print(number_of_tweets)
        except TimeoutException:
            # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:
            print(number_of_tweets)
            break

    # extract the html for the whole lot:
    page_source = driver.page_source

except TimeoutException:
    print(number_of_tweets)

20
40
60
80
100
120
140
160
180
200


In [40]:
soup = bs4.BeautifulSoup(page_source)

tweets = []
for li in soup.find_all("li", class_='js-stream-item'):

    # If our li doesn't have a tweet-id, we skip it as it's not going to be a tweet.
    if 'data-item-id' not in li.attrs:
        continue

    else:
        tweet = {
            'tweet_id': li['data-item-id'],
            'text': None,
            'user_id': None,
            'user_screen_name': None,
            'user_name': None,
            'created_at': None,
            'retweets': 0,
            'likes': 0,
            'replies': 0
        }

    # Tweet Text
    text_p = li.find("p", class_="tweet-text")
    if text_p is not None:
        tweet['text'] = text_p.get_text()

    # Tweet User ID, User Screen Name, User Name
    user_details_div = li.find("div", class_="tweet")
    if user_details_div is not None:
        tweet['user_id'] = user_details_div['data-user-id']
        tweet['user_screen_name'] = user_details_div['data-screen-name']
        tweet['user_name'] = user_details_div['data-name']

    # Tweet date
    date_span = li.find("span", class_="_timestamp")
    if date_span is not None:
        tweet['created_at'] = float(date_span['data-time-ms'])

    # Tweet Retweets
    retweet_span = li.select("span.ProfileTweet-action--retweet > span.ProfileTweet-actionCount")
    if retweet_span is not None and len(retweet_span) > 0:
            tweet['retweets'] = int(retweet_span[0]['data-tweet-stat-count'])

    # Tweet Likes
    like_span = li.select("span.ProfileTweet-action--favorite > span.ProfileTweet-actionCount")
    if like_span is not None and len(like_span) > 0:
        tweet['likes'] = int(like_span[0]['data-tweet-stat-count'])

    # Tweet Replies
    reply_span = li.select("span.ProfileTweet-action--reply > span.ProfileTweet-actionCount")
    if reply_span is not None and len(reply_span) > 0:
        tweet['replies'] = int(reply_span[0]['data-tweet-stat-count'])

    tweets.append(tweet)

C:\Users\Ben\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Ben\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [41]:
tmp = pd.DataFrame(tweets)
tmp['created_at'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x/1000)))
tmp.head()

,created_at,likes,replies,retweets,text,tweet_id,user_id,user_name,user_screen_name
0,1.430525e+12,4,0,10002,#DNotes co-founder Alan Yong will be attending...,594290197749194752,2306894348,DNotes,DNotesCoin
1,1.430525e+12,0,0,0,Helllllsssssss yeaaaaa!!!!!!! Nicki here I com...,594290166468112384,264383298,SHOORRTTYYY!!!,BOOM_BTC
2,1.430525e+12,0,0,0,Bitcoin : quelle réalité pour les utilisateurs...,594290119684845568,3152422890,CAWAM,CAWAMcom
3,1.430525e+12,0,0,0,New post: Bitcoin Slowly Getting into Mainstre...,594290086281228288,2387831629,CoinBorse NEWS,CoinBorse
4,1.430525e+12,0,0,0,New post: BTC - 247 http://hodl.cz,594290083072581632,2387831629,CoinBorse NEWS,CoinBorse


In [204]:
time1 = datetime.datetime(2019,1,1,10,20)
time2 = datetime.datetime(2019,1,1,11,50)

interval_mins=20
tmp = pd.date_range(start = time1, end = time2, freq = str(interval_mins)+'min')
tmp[0].isoformat()

'2019-01-01T10:20:00'

In [268]:
datetime.datetime.utcfromtimestamp(1563704457)

datetime.datetime(2019, 7, 21, 10, 20, 57)

In [232]:
tmp = pd.datetime.today().date()
datetime.datetime(tmp.year,tmp.month,tmp.day)

datetime.datetime(2019, 7, 20, 0, 0)

In [207]:
tweets_df= pd.DataFrame(data=np.array([tweet_texts,dates,user_names,user_ids]).T,
                 columns = ['text','created_at','user_name','user_id'])
            
tweets_df['followers'] = tweets_df['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])

In [208]:
tweets_df.to_csv(r'C:\Users\Ben\Documents\2019 Summer\TradingAlgo\Tweets\temp\bitcoin.csv')

,text,created_at,user_name,user_id,followers
0,#DNotes co-founder Alan Yong will be attending...,2015-05-01 23:59:59,DNotesCoin,2306894348,7088
1,Helllllsssssss yeaaaaa!!!!!!! Nicki here I com...,2015-05-01 23:59:51,BOOM_BTC,264383298,628
2,Bitcoin : quelle réalité pour les utilisateurs...,2015-05-01 23:59:40,CAWAMcom,3152422890,876
3,New post: Bitcoin Slowly Getting into Mainstre...,2015-05-01 23:59:32,CoinBorse,2387831629,109
4,New post: BTC - 247 http://hodl.cz,2015-05-01 23:59:31,CoinBorse,2387831629,109


In [144]:
# body = browser.find_element_by_tag_name('body')
# reached_bottom = False
# # While not reached_bottom:
# for i in range(10):
#     body.send_keys(Keys.PAGE_DOWN)
#     print(body.size)
#     time.sleep(0.001)
# #     if:
# #         reached_bottom=True
    

tweets = browser.find_elements_by_class_name('js-actionable-tweet')
tweet_texts = [tweet.find_element_by_class_name('js-tweet-text-container').text for tweet in tweets]
user_names = [tweet.get_attribute('data-screen-name') for tweet in tweets]
user_ids = [tweet.get_attribute('data-user-id') for tweet in tweets]
timestamps = [tweet.find_element_by_class_name('js-short-timestamp').get_attribute('data-time') for tweet in tweets]
dates = [datetime.datetime.utcfromtimestamp(int(ts)) for ts in timestamps]

# tweets[0].find_element_by_class_name('username').text


In [150]:
tmp= pd.DataFrame(data=np.array([tweet_texts,dates,user_names,user_ids]).T,
                 columns = ['text','created_at','user_name','user_id'])
tmp['followers'] = tmp['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])
tmp.head()

In [146]:
len(dates)

220

In [ ]:
# initial_height = browser.execute_script("return document.body.scrollHeight")
# print(initial_height)
# while browser.execute_script("return document.body.scrollHeight") ==initial_height:
#     body.send_keys(Keys.PAGE_DOWN)
#     time.sleep(0.5)
#     print(browser.execute_script("return document.body.scrollHeight"))


In [151]:
tmp= pd.DataFrame(data=np.array([tweet_texts,dates,user_names,user_ids]).T,
                 columns = ['text','created_at','user_name','user_id'])
tmp['followers'] = tmp['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])
tmp.head()

,text,created_at,user_name,user_id
0,#DNotes co-founder Alan Yong will be attending...,2015-05-01 23:59:59,DNotesCoin,2306894348
1,Helllllsssssss yeaaaaa!!!!!!! Nicki here I com...,2015-05-01 23:59:51,BOOM_BTC,264383298
2,Bitcoin : quelle réalité pour les utilisateurs...,2015-05-01 23:59:40,CAWAMcom,3152422890
3,New post: Bitcoin Slowly Getting into Mainstre...,2015-05-01 23:59:32,CoinBorse,2387831629
4,New post: BTC - 247 http://hodl.cz,2015-05-01 23:59:31,CoinBorse,2387831629


In [153]:
tmp.iloc[2,0]

'Bitcoin : quelle réalité pour les utilisateurs ? - ITespresso.fr  #bitcoin http://bit.ly/1JfvM12 '

In [9]:
class Tweet_Collecter(object):
    
    def __init__(self, init_browser=True, browser=None):
        
        import os
        import math
        import pandas as pd
        import numpy as np
        from selenium.webdriver.chrome.options import Options
        from selenium import webdriver
        import tweepy
        import time
        import datetime
        import requests
        
        self.init_browser= init_browser
        
        self.C_Key = 'ILYTd5Abkw85OKTd5sAbSpPdC'
        self.C_Secret='1hHTjqyq5Kitt3b5gka6uPMkvKzuz5wO7C63HcRLP5v2mHUtz6'
        self.A_Token='2758135350-UfWPEgJPQJTCHvCQRQEzcavyl45mcwwLkRnWWBi'
        self.A_Token_Secret='LKa0IlzGhWdLKgzhATUtFx0kM5AGG6lAWBUoOEP1lKj9g'

        auth = tweepy.OAuthHandler(consumer_key=C_Key, consumer_secret=C_Secret)
        auth.set_access_token(A_Token, A_Token_Secret)
        self.auth = auth
        
        self.api = tweepy.API(auth)
        if init_browser:
            chrome_option = Options()
            chrome_option.add_argument(r'user-data-dir=C:\Users\Ben\AppData\Local\Google\Chrome\User Data\Default')
            chrome_option.add_argument("--no-sandbox")
            chrome_option.add_argument("--disable-dev-shm-usage")
            chrome_option.add_argument("--headless")
            self.browser = webdriver.Chrome(executable_path=r'C:\Users\Ben\Documents\Varsity Surf\chromedriver_win32\chromedriver.exe',
                                            chrome_options=chrome_option)
        else:
            self.browser = browser
    
    def collect_tweets(self, filters = ['#btc', 'bitcoin'], start_date=datetime.datetime.today()-datetime.timedelta(days=1),
                       end_date= datetime.datetime.today(), approx_tweets_per_interval = 200, interval_mins=20,
                      save_folder_name='temp', filename='bitcoin', round_interval = True):
    
        browser = self.browser
        api = self.api
        
        if round_interval:
            start_date = start_date - datetime.timedelta(minutes=start_date.minute % interval_mins,
                                 seconds= start_date.second,
                                 microseconds=start_date.microsecond)

        if start_date>end_date:
            raise Exception('Start date must be after end date')

        print('Calculating tweet ids to be collected...')
        ################### Find id of tweet exactly 24 hours later
        since_datetime = start_date.date()
        since_datetime = datetime.datetime(since_datetime.year, since_datetime.month, since_datetime.day)
        until_datetime = (start_date + datetime.timedelta(days=1)).date()
        until_datetime = datetime.datetime(until_datetime.year, until_datetime.month, until_datetime.day)
        end_datetime = end_date.date()
        end_datetime = datetime.datetime(end_datetime.year, end_datetime.month, end_datetime.day)

        since_date = since_datetime.isoformat()[:10]
        until_date = until_datetime.isoformat()[:10]


        filters = [fil.replace('#','%23') for fil in filters]
        self.filters = filters
        query_filters = str('%20OR%20'.join(filters))
        query = query_filters + '%20since%3A'+since_date+'%20until%3A'+until_date+'&src=typd'

        base_url = 'https://twitter.com/search?f=tweets&q='
        url = base_url + query
        browser.get(url) 
        time.sleep(1)


        endtweet_id = int(browser.find_elements_by_class_name('js-actionable-tweet')[0].get_attribute('data-tweet-id'))

        #################### Find id of last tweet before timeperiod starts

        until_datetime2 = start_date.date()
        until_datetime2 = datetime.datetime(until_datetime2.year, until_datetime2.month, until_datetime2.day)
        until_date2 = until_datetime2.isoformat()[:10]

        query = query_filters + '%20until%3A'+until_date2+'%20lang%3Aen&src=typd'

        url = base_url + query

        browser.get(url) 
        time.sleep(1)

        starttweet_id = int(browser.find_elements_by_class_name('js-actionable-tweet')[0].get_attribute('data-tweet-id'))

        #################### calculate tweet id pairs needed to collect correct tweets at each sub-timeperiod


        id_interval = (endtweet_id - starttweet_id)/((60*24)/interval_mins)

        ##calculate which id to start on and end on
        ##start tweet
        start_timejump = start_date - since_datetime #difference between time specified to start at and time at which
                                               #starttweet_id currently specifies to collect from
        start_intervalsjump = int(math.ceil(start_timejump.total_seconds()/(60*interval_mins)))


        total_timeperiod_length = end_date - start_date #total length of time over whole specified timeperiod
                                                        # enables calculation of how many id intervals need to be collected

        num_intervals= math.ceil(total_timeperiod_length.total_seconds()/(60*interval_mins)) #number of intervals to collect

        start_id = int(starttweet_id + start_intervalsjump * id_interval)

        since_ids= [int(start_id + i*id_interval) for i in range(num_intervals)]
        max_ids = [int(x + id_interval) for x in since_ids]
        
        time_range = pd.date_range(start = start_date, end = end_date, freq = str(interval_mins)+'min')

        print('Tweet ids calculated!')

        ################## Start tweet collection
        filepath_stem = r'C:\Users\Ben\Documents\2019 Summer\TradingAlgo\Tweets\ '.strip() + save_folder_name + r'\ '.strip()       
        if not os.path.exists(filepath_stem):
            os.mkdir(filepath_stem)
    
        print('Beginning tweet collection:')
        iteration_counter=1

        for since_id, max_id, timestamp in zip(since_ids,max_ids, time_range): # for each subtimeperiod/interval of tweets

            print('Collecting tweets from interval:',iteration_counter)
            iteration_counter+=1

            query = query_filters + '%20since_id%3A'+str(int(since_id))+'%20max_id%3A'+str(int(max_id))+'&src=typd'
            url = base_url + query

            browser.get(url) 
            time.sleep(1)

            #perform browser scrolling to load all tweets
            SCROLL_PAUSE_TIME = 0.5

            # Get scroll height
            last_height = browser.execute_script("return document.body.scrollHeight")

            counter=0
            while len(browser.find_elements_by_class_name('js-actionable-tweet'))<approx_tweets_per_interval:
                print(len(browser.find_elements_by_class_name('js-actionable-tweet')))
                # Scroll down to bottom

                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    counter +=1
                    print('Timeout:',counter)
                    if counter==5:
                        break
                else:
                    counter=0
                last_height = new_height

            ### scrolling complete

            #collect tweets and data
            tweets = browser.find_elements_by_class_name('js-actionable-tweet')
            
            tweet_texts = []
            user_names = []
            user_ids = []
            timestamps = []
            dates = []
            for tweet in tweets:    
                try:
                    tweet_texts.append(tweet.find_element_by_class_name('js-tweet-text-container').text)
                    user_names.append(tweet.get_attribute('data-screen-name'))
                    user_ids.append(tweet.get_attribute('data-user-id'))
                    timestamps.append(tweet.find_element_by_class_name('js-short-timestamp').get_attribute('data-time'))
                except:
                    continue
            
            dates = [datetime.datetime.utcfromtimestamp(int(ts)) for ts in timestamps]
            
#                 tweet_texts = [tweet.find_element_by_class_name('js-tweet-text-container').text for tweet in tweets]
#                 user_names = [tweet.get_attribute('data-screen-name') for tweet in tweets]
#                 user_ids = [tweet.get_attribute('data-user-id') for tweet in tweets]
#                 timestamps = [tweet.find_element_by_class_name('js-short-timestamp').get_attribute('data-time') for tweet in tweets]
#                 dates = [datetime.datetime.utcfromtimestamp(int(ts)) for ts in timestamps]

            ### convert data to dataframe

            tweets_df= pd.DataFrame(data=np.array([tweet_texts,dates,user_names,user_ids]).T,
                 columns = ['text','created_at','user_name','user_id'])

            #tweets_df['followers'] = tweets_df['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])
            savefile = filepath_stem + filename + timestamp.isoformat()[:19].replace(':','') +'.csv'

            tweets_df.to_csv(savefile)

        print('Finished!')
        return self

In [154]:
tmp['followers'] = tmp['user_id'].apply(lambda x: api.get_user(x)._json['followers_count'])

In [155]:
tmp.tail()

,text,created_at,user_name,user_id,followers
215,Tsinghuaイイね。この辺りの動きの軽さは素晴らしい。 https://bitcoinm...,2015-05-01 23:42:19,toshi_kaseda,2941941391,1037
216,#Bitcoin Circle Building Its Global Vision Tha...,2015-05-01 23:42:01,CoinfeedIO,2961909107,1336
217,My robot is level 3 and I've earned a total of...,2015-05-01 23:41:51,ChrissCoin,3121596999,13
218,Bitcoin dood? Goldman Sachs gelooft er nog in ...,2015-05-01 23:41:30,CAWAMcom,3152422890,876
219,Obsolete ASIC mining... useful to prevent 51% ...,2015-05-01 23:41:23,bonniemck3,2809127089,338


In [156]:
tmp.shape

(220, 5)